In [1]:
import os
import keras
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten, Dropout
from keras.layers.core import Dense
from keras.datasets import mnist
from keras.optimizers import Adam
from keras.callbacks import TensorBoard

Using TensorFlow backend.
/Users/a1900001.1/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
def lenet(input_shape, num_classes):
    model = Sequential()
    model.add(Conv2D(20, kernel_size=5, padding="same",
                    input_shape=input_shape, activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(50, kernel_size=5, padding="same",
                    activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Flatten())
    model.add(Dense(500, activation="relu"))
    model.add(Dense(num_classes))
    model.add(Activation("softmax"))
    return model

In [8]:
class MNISTDataset():
    def __init__(self):
        self.image_shape = (28,28,1)
        self.num_classes = 10
        
    def get_batch(self):
        (x_train, y_train), (x_test, y_test) = mnist.load_data()
        
        x_train, x_test = [self.preprocess(d) for d in [x_train, x_test]]
        y_train, y_test = [self.preprocess(d, label_data=True) for d in [y_train, y_test]]
        
        return x_train, y_train, x_test, y_test
    
    def preprocess(self, data, label_data=False):
        if label_data:
            data = keras.utils.to_categorical(data, self.num_classes)
        else:
            data = data.astype("float32")
            data /= 255
            shape = (data.shape[0], ) + self.image_shape
            data = data.reshape(shape)
            
        return data

In [13]:
class Trainer():
    def __init__(self, model, loss, optimizer):
        self._target = model
        self._target.compile(loss=loss, optimizer=optimizer, metrics=["accuracy"])
        self.verbose = 1
        self.log_dir = os.path.join(os.path.dirname('__file__'), 'logdir')
    
    def train(self, x_train, y_train, batch_size, epochs, validation_split):
        if os.path.exists(self.log_dir):
            import shutil
            shutil.rmtree(self.log_dir)
        os.mkdir(self.log_dir)
        
        self._target.fit(
            x_train, y_train, batch_size=batch_size, epochs=epochs,
            validation_split=validation_split, callbacks=[TensorBoard(log_dir=self.log_dir)],
            verbose=self.verbose
        )

In [15]:
dataset = MNISTDataset()

model = lenet(dataset.image_shape, dataset.num_classes)

x_train, y_train, x_test, y_test = dataset.get_batch()
trainer = Trainer(model, loss="categorical_crossentropy", optimizer=Adam())
trainer.train(x_train, y_train, batch_size=128, epochs=12, validation_split=0.2)

score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Train on 48000 samples, validate on 12000 samples
Epoch 1/12
48000/48000 [==============================] - 88s 2ms/step - loss: 0.1777 - acc: 0.9474 - val_loss: 0.0600 - val_acc: 0.9827
Epoch 2/12
48000/48000 [==============================] - 129s 3ms/step - loss: 0.0485 - acc: 0.9850 - val_loss: 0.0516 - val_acc: 0.9847
Epoch 3/12
48000/48000 [==============================] - 111s 2ms/step - loss: 0.0324 - acc: 0.9896 - val_loss: 0.0373 - val_acc: 0.9889
Epoch 4/12
48000/48000 [==============================] - 121s 3ms/step - loss: 0.0241 - acc: 0.9925 - val_loss: 0.0359 - val_acc: 0.9895
Epoch 5/12
48000/48000 [==============================] - 147s 3ms/step - loss: 0.0164 - acc: 0.9945 - val_loss: 0.0380 - val_acc: 0.9897
Epoch 6/12
48000/48000 [==============================] - 134s 3ms/step - loss: 0.0145 - acc: 0.9951 - val_loss: 0.0424 - val_acc: 0.9880
Epoch 7/12
48000/48000 [==============================] - 149s 3ms/step - loss: 0.0112 - acc: 0.9961 - val_loss: 0.0413 - v